In [ ]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Ranking and reranking with discoveryengine API

* Rank and rerank documents with RAG
*  https://cloud.google.com/generative-ai-app-builder/docs/ranking#genappbuilder_rank-python

In [4]:
# @title Install packages
!pip install --upgrade --user --quiet google-cloud-aiplatform \
                                      google-cloud-discoveryengine

## Set configuration

In [5]:
# @title Authentication to access to GCP

# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

In [6]:
# @title Define constants
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

In [7]:
# @title Import libraries
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession
)

from vertexai.preview.generative_models import grounding

In [ ]:
# @title Initialize Vertex AI

# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)

# https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerativeModel
model = GenerativeModel(MODEL_NAME)

### Define helper function

In [15]:
from google.cloud import discoveryengine_v1alpha as discoveryengine

re_rank_client = discoveryengine.RankServiceClient()
ranking_config = re_rank_client.ranking_config_path(
    project=PROJECT_ID,
    location="global",
    ranking_config="default_ranking_config",
)

def re_rank(query, response_list):

  records = []
  top_n = 20

  # building records to rank
  for index, response in enumerate(response_list):

    title = response
    content = response
    records.append(discoveryengine.RankingRecord(id=str(index), title=title, content=content))

  # building request
  request = discoveryengine.RankRequest(
      ranking_config = ranking_config,
      model = "semantic-ranker-512@latest",
      top_n = top_n,
      query = query,
      records = records
    )

  # execute request
  re_ranked_response = re_rank_client.rank(request=request)

  return re_ranked_response

ImportError: cannot import name 'discoveryengine_v1alpha' from 'google.cloud' (unknown location)

In [ ]:
query = "Gemini"
outcome_list = ["The Gemini zodiac symbol often depicts two figures standing side-by-side.",
                "Gemini is a cutting edge large language model created by Google.",
                "Gemini is a constellation that can be seen in the night sky."
                ]

re_rank(query, outcome_list)

records {
  id: "1"
  title: "Gemini is a cutting edge large language model created by Google."
  content: "Gemini is a cutting edge large language model created by Google."
  score: 0.68
}
records {
  id: "2"
  title: "Gemini is a constellation that can be seen in the night sky."
  content: "Gemini is a constellation that can be seen in the night sky."
  score: 0.67
}
records {
  id: "0"
  title: "The Gemini zodiac symbol often depicts two figures standing side-by-side."
  content: "The Gemini zodiac symbol often depicts two figures standing side-by-side."
  score: 0.53
}